In [130]:
#First import data and filter to the 'text' column
import pandas as pd

#all years
data = pd.read_json('test.json')
df = pd.DataFrame(data)

##Data cleaning
df['text'] = df['text'].astype(str).replace(r'\\n',' ', regex=True)
df['text'] = df['text'].astype(str).replace(r'<','', regex=True)
df['text'] = df['text'].astype(str).replace(r'>','', regex=True)
df['text'] = df['text'].astype(str).replace(r'. ',' ', regex=True)
df['text'] = df['text'].astype(str).replace(r'\[','', regex=True)
df['text'] = df['text'].astype(str).replace(r'\]','', regex=True)
df['text'] = df['text'].astype(str).replace(r'\'','', regex=True)

#young years
youth = "1869, 1870, 1871, 1872, 1873, 1874, 1875"
df_youth = df[df['year']<=1875]

#turning point
turningP = "1876, 1877, 1878"


#maturity
mature = "1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888"
df_mature = df[df['year']>=1879]

df_mature.to_csv('df_mature.csv')


In [105]:
documents = data['text']

from pprint import pprint #pretty-printer
from collections import defaultdict
from nltk.corpus import stopwords

#remove common words
stoplist = stopwords.words('german')
#additional stopwords
stopwords_new = ['.', 'ii.']
stoplist.append(stopwords_new) #if I need to add more stopwords manually

#tokenize
texts = [
    [word for word in document.lower().split() if word not in stoplist] 
    for document in documents
]

#remove words that appear only once

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [
    [token for token in text if frequency[token] > 1]
    for text in texts
]

In [102]:
from gensim import corpora

dictionary = corpora.Dictionary(texts)
dictionary.save('/tmp/document1.dict')

dictionary.token2id


{"'": 0,
 "'aeschylu": 1,
 "'auc": 2,
 "'da": 3,
 "'de": 4,
 "'di": 5,
 "'e": 6,
 "'ei": 7,
 "'einflu": 8,
 "'euripide": 9,
 "'hef": 10,
 "'i": 11,
 "'ic": 12,
 "'ii": 13,
 "'is": 14,
 "'lessin": 15,
 "'mange": 16,
 "'nac": 17,
 "'schlege": 18,
 "'seh": 19,
 "'si": 20,
 "'socrate": 21,
 "'sokrate": 22,
 "'sophocle": 23,
 "'wa": 24,
 "'wi": 25,
 "'wichtig": 26,
 "'wunderbar": 27,
 "'zu": 28,
 "'„di": 29,
 '(i': 30,
 ".'": 31,
 '1': 32,
 '16te': 33,
 'a': 34,
 'abe': 35,
 'absolut': 36,
 'absolute': 37,
 'adras': 38,
 'aeschyleische': 39,
 'aeschylu': 40,
 'aestheti': 41,
 'aktio': 42,
 'al': 43,
 'all': 44,
 'allei': 45,
 'allgemeine': 46,
 'allmählic': 47,
 'alte': 48,
 'altenglische': 49,
 "alterthum.'": 50,
 'anblic': 51,
 'andant': 52,
 'andr': 53,
 'anschauun': 54,
 'antik': 55,
 'antike': 56,
 'ar': 57,
 'aristophane': 58,
 'art': 59,
 'attische': 60,
 'au': 61,
 'auc': 62,
 'aufführun': 63,
 'aufgab': 64,
 'auge': 65,
 'ausdruc': 66,
 'ausdruck': 67,
 'auseinandergerissene': 68,


In [107]:
import gensim

model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)
model.build_vocab(texts)

AttributeError: 'list' object has no attribute 'words'